In [379]:
import os
import json
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

from typing import *
from matplotlib import pyplot as plt
import plotly.express as px

#%%

df = pd.read_excel(
    'DataSet_EKB_200000.xlsx',
    engine='openpyxl',
)

#%%

df['Другая продукция в контрактах'] = (
    df['Другая продукция в контрактах']
    .progress_apply(
        lambda x: (
            x.rsplit('{', maxsplit=1)[0][:-1] + ']'
            if not x.endswith(']') else x
        )
        if not pd.isnull(x) else None
    )
)

#%%

def parse_json(s: str) -> list:
    if not pd.notnull(s):
        result = None
    else:
        try:
            result = json.loads(s)
        except json.JSONDecodeError:
            return None

    return result

json_columns = [
    'Характеристики СТЕ',
    'Регионы поставки',
    'Поставщики',
    'Другая продукция в контрактах',
    'Цена'
]
for col in tqdm(json_columns):
    df[col] = df[col].apply(parse_json)

df.head()

#%%

df = df.set_index('Идентификатор СТЕ')
df.head()

#%%
#%%

100%|██████████| 5/5 [00:37<00:00,  7.41s/it]


,Наименование СТЕ,Категория,Описание,Характеристики СТЕ,Регионы поставки,Кол-во заключенных контрактов,Поставщики,Страна происхождения,Другая продукция в контрактах,Просмотры,Идентификатор КПГЗ,Код КПГЗ,Модель,Цена
Идентификатор СТЕ,,,,,,,,,,,,,,
20528973,"Светодиодная лента 2835, ULTRA, 12 Вольт, 60 l...",Лампы светодиодные,NaN,"[{'Name': 'Цвет свечения', 'Id': -506425117, '...",[{'Name': 'Москва'}],NaN,"[{'SupplierId': 2310783, 'Name': 'ООО ""ЛЮМЕНВА...",NaN,None,1,12106442.0,01.10.05.02.04,2835ULTRA,None
34860341,Светодиодный светильник Geniled ЛПО 1200?180?...,Настенно-потолочные светильники,NaN,"[{'Name': 'Вид климатического исполнения', 'Id...","[{'Name': 'Свердловская'}, {'Name': 'Москва'}]",1.0,"[{'SupplierId': 1559525, 'Name': 'Общество с о...",РОССИЯ,None,41,12114082.0,01.10.05.03.01.05,ЛПО 1200?180?40 40Вт 5000К Опал,None
34172198,Елка пристенно-угловая Графская премиум 210 см...,Предметы внутреннего интерьера,NaN,"[{'Name': 'Цвет', 'Id': 340085190, 'Value': 'З...","[{'Name': 'Москва'}, {'Name': 'Все регионы'}]",NaN,"[{'SupplierId': 1348988, 'Name': 'ИП Чепало Ок...",NaN,None,5,68093.0,01.20.08,Елка пристенно-угловая Графская премиум 210 см.,None
1318014,Картридж HP CE278A/Canon 728 LJ P1560/1566/160...,Расходные материалы и комплектующие для лазерн...,NaN,"[{'Name': 'Тип расходника', 'Id': 317455168, '...","[{'Name': 'Москва'}, {'Name': 'Московская'}]",9.0,"[{'SupplierId': 1111141, 'Name': 'Общество с о...",NaN,"[{'OtherSkuId': 1398583, 'OtherSkuName': '013R...",6,12105204.0,01.13.13.01.01,SF-CE278(2K),None
1251411,Штамп самонаборный Pr.C20/3-Set 38х14 (аналог ...,Штемпели,NaN,"[{'Name': 'Наличие рамки', 'Id': 366100137, 'V...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",71.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",АВСТРИЯ,"[{'OtherSkuId': 1171023, 'OtherSkuName': 'Berl...",72,12120460.0,01.15.06.07,Штамп самонаборный пласт. 3стр. Pr.C20/3-Set 3...,"[{'Cost': 454.67}, {'Cost': 620.0}, {'Cost': 6..."


![title](./item_page.jpg)

![title](./search2.jpg)

![title](./empty.jpg)

In [468]:
df[df['hasSimilars'] == 0].reset_index()[['Идентификатор СТЕ', 'Наименование СТЕ', 'Категория', 'Другая продукция в контрактах']].iloc[2]

Идентификатор СТЕ                                                         34172198
Наименование СТЕ                 Елка пристенно-угловая Графская премиум 210 см...
Категория                                           Предметы внутреннего интерьера
Другая продукция в контрактах                                                 None
Name: 2, dtype: object

In [446]:
df[df['hasSimilars'] == 1].reset_index().iloc[find_neighbors_by_tfidf_neighbor(2)].head(5)

,Идентификатор СТЕ,Наименование СТЕ,Категория,Описание,Характеристики СТЕ,Регионы поставки,Кол-во заключенных контрактов,Поставщики,Страна происхождения,Другая продукция в контрактах,Просмотры,Идентификатор КПГЗ,Код КПГЗ,Модель,Цена,session_ids,item_id,hasSimilars,indice
44502,34318478,"Искусственная елка СОФИЯ 0,6 м, Ели PENERI",Предметы внутреннего интерьера,NaN,"[{'Name': 'Материал', 'Id': 345373325, 'Value'...",[{'Name': 'Москва'}],2.0,"[{'SupplierId': 1286028, 'Name': 'Лебедева Ири...",РОССИЯ,"[{'OtherSkuId': 34319911, 'OtherSkuName': 'Бор...",0,68093.0,01.20.08,"СОФИЯ 0,6 м",None,"[[21186002, 34332942, 34319897, 34319850, 3431...",[34318478],1,167113
34349,19692866,"Елочная верхушка ЗВЕЗДА ДЕЛЮКС, пластик, глитт...",Предметы внутреннего интерьера,NaN,"[{'Name': 'Вид продукции', 'Id': -553077661, '...","[{'Name': 'Москва'}, {'Name': 'Московская'}]",4.0,"[{'SupplierId': 1286028, 'Name': 'Лебедева Ири...",NaN,"[{'OtherSkuId': 20375307, 'OtherSkuName': 'Бор...",0,68093.0,01.20.08,Елочная верхушка,None,"[[20382453, 20378930, 20451532, 20454836, 2041...",[19692866],1,117842
41323,20968751,Верхушка на елку Снежинка Сияющая золотая,Предметы внутреннего интерьера,NaN,"[{'Name': 'Материал', 'Id': -484412139, 'Value...","[{'Name': 'Москва'}, {'Name': 'Московская'}]",3.0,"[{'SupplierId': 1110975, 'Name': 'ОБЩЕСТВО С О...",NaN,"[{'OtherSkuId': 20559558, 'OtherSkuName': 'Вен...",0,68093.0,01.20.08,Верхушка на елку Снежинка Сияющая золотая,None,"[[34505014, 34327902, 20390929, 20513687, 2096...",[20968751],1,150171
23959,20167409,Наклейка новогодняя Елка,Предметы внутреннего интерьера,NaN,"[{'Name': 'Материал', 'Id': -528613991, 'Value...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",1.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",NaN,"[{'OtherSkuId': 20517536, 'OtherSkuName': 'АЖУ...",0,68093.0,01.20.08,Наклейка новогодняя Елка,None,"[[20355284, 20517536, 20553173, 20419635, 2031...",[20167409],1,81466
37992,20810900,Гирлянда Сетка,Предметы внутреннего интерьера,NaN,"[{'Name': 'Цвет свечения', 'Id': -490287020, '...","[{'Name': 'Москва'}, {'Name': 'Московская'}]",3.0,"[{'SupplierId': 1271402, 'Name': 'Индивидуальн...",NaN,"[{'OtherSkuId': 34316518, 'OtherSkuName': 'Бум...",0,68093.0,01.20.08,Сетка,None,"[[34316758, 20895384, 34316756, 21060603, 3431...",[20810900],1,133708


In [355]:
#from tqdm.notebook import tqdm
#tqdm.pandas()
def shuffle(v):
    np.random.shuffle(v)
    return v
def drop_duplicates(x):
    new_session = []
    x = list(filter(lambda y: len(y[0]) > 0, x))
    x = sorted(x, key=lambda x: x[0])
    for i in range(len(x)-1):
        if len(x[i][0][0]) > 0 and x[i][0][0] != x[i+1][0][0]:
            new_session.append(x[i][1])
    if len(x) > 0 and len(x[-1][0]) > 0:
        if len(new_session) > 0 and x[-1][1] != new_session[-1]:
            new_session.append(x[-1][1])
        elif len(new_session) == 0:
            new_session.append(x[-1][1])
    return new_session
        
def shuffle_tuple(v):
    indices = shuffle(np.arange(len(v)))
    ids = []
    item_texts = []
    for _, id_ in v:
        ids.append(id_)
    for t, _ in v:
        item_texts.append(t)
    ids = np.array(ids)[indices]
    item_texts = np.array(item_texts)[indices]
    
    return list(zip(item_texts, ids))

def multiple_samples(v):
    num_samples = min(int(np.log2(len(v))), 8)
    samples = [shuffle(drop_duplicates(v))]
    for i in range(4, num_samples):
        samples.append(shuffle(drop_duplicates(v.copy())))
    return samples

items = items.progress_apply(multiple_samples).explode()

  0%|          | 0/151894 [00:00<?, ?it/s]

In [356]:
orig_item = pd.Series(items.index.values)
orig_item.index = items.index.values
items = items + orig_item.apply(lambda x: [x])

In [380]:
import numpy as np

def by_only_ids(df):
    df['session_ids'] = df['Другая продукция в контрактах'].dropna().apply(lambda x: [item['OtherSkuId'] for item in x])
    df['item_id'] = df.index
    df['item_id'] = df['item_id'].apply(lambda x: [x])
    df['session_ids'] = df['item_id'] + df['session_ids']
    df['session_ids'] = df['session_ids'].dropna().apply(lambda v: [shuffle(v)] + [shuffle(v.copy()) for i in range(5, int( np.log2(len(v))))])

    return df 

df = by_only_ids(df)

In [383]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=list(items.values), vector_size=100, window=5, min_count=10, workers=4, negative=10)

model.train(items.values, total_examples=len(items.values), epochs=10)

(48122860, 52792340)

In [384]:
existed_index = df[df.index.isin(model.wv.index_to_key)].index
existed_index = list(existed_index.values)

In [385]:
#most_similar = model.wv.keys()
embeddings = []
model_keys = list(existed_index)
for key in model_keys:
    embeddings.append(model.wv[key])
df_no_index = df.loc[existed_index].reset_index()

In [386]:
import faiss
def fit_search_index(vectors):
    index = faiss.index_factory(100, "IVF100,Flat")

    index.train(vectors) # вектора, которые могут быть соседями
    index.add(vectors) # вектора, которые могут быть соседями
    return index
index = fit_search_index(np.array(embeddings))
def similars(index, vectors, top_k):
    scores, neighbors = index.search(vectors, top_k)
    return neighbors, scores

neighbors, scores = similars(index, np.array(embeddings), 20)
similar_index_to_ste = {i:existed_index[i] for i in range(len(existed_index))}
ste_to_similar_index = {existed_index[i]:i for i in range(len(existed_index))}

In [447]:
df_no_index.iloc[list(neighbors[1])].head(5)

,Идентификатор СТЕ,Наименование СТЕ,Категория,Описание,Характеристики СТЕ,Регионы поставки,Кол-во заключенных контрактов,Поставщики,Страна происхождения,Другая продукция в контрактах,Просмотры,Идентификатор КПГЗ,Код КПГЗ,Модель,Цена,session_ids,item_id
1,1228720,Блокнот на спирали А5 60 листов 40 штук в упак...,Блокнот,NaN,"[{'Name': 'Отверстия для прошивки', 'Id': 3656...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",63.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",РОССИЯ,"[{'OtherSkuId': 1180614, 'OtherSkuName': 'ArtS...",121,66924.0,01.15.02.01,STENOGRAPHY 6с 10 клетка,"[{'Cost': 23.14}, {'Cost': 23.19}, {'Cost': 24...","[[1248169, 1227450, 1227404, 1248326, 1185008,...",[1228720]
361,1247785,Степлер ATTACHE 8215 (N24/6&26/6) до 25 лист. ...,Степлеры (сшиватели),NaN,"[{'Name': 'Цвет корпуса', 'Id': 299368878, 'Va...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",30.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",КИТАЙ,"[{'OtherSkuId': 1179412, 'OtherSkuName': 'Albi...",32,12104724.0,01.15.05.02.01,NaN,"[{'Cost': 155.11}, {'Cost': 184.87}, {'Cost': ...","[[1248094, 17927810, 1249120, 1237495, 1791911...",[1247785]
6449,1237057,Этикетки самоклеящиеся 3425 универсал. 105х57м...,Этикетки самоклеящиеся,NaN,"[{'Name': 'Шаблон для скачивания', 'Id': 29112...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",1.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",ГЕРМАНИЯ,"[{'OtherSkuId': 1227434, 'OtherSkuName': 'Блок...",1,12104668.0,01.15.01.09,NaN,[{'Cost': 2733.93}],"[[17872802, 1232129, 16484823, 1226949, 179535...",[1237057]
3912,1231786,"Дырокол SAX 506 до 40л., металл, с лин., синий...",Дыроколы канцелярские,NaN,"[{'Name': 'Особенности', 'Id': 291064782, 'Val...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",5.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",NaN,"[{'OtherSkuId': 1171063, 'OtherSkuName': 'Berl...",6,12114402.0,01.15.05.02.05.01,NaN,[{'Cost': 1730.0}],"[[1232291, 1230164, 1227444, 1190254, 17796685...",[1231786]
181,20001081,Набор текстовыделителей Attache Economy Uno (т...,Маркеры,NaN,"[{'Name': 'Форма наконечника', 'Id': 365683510...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",90.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",КИТАЙ,"[{'OtherSkuId': 34137737, 'OtherSkuName': '""VK...",123,12104687.0,01.15.04.02.04,892707,"[{'Cost': 70.38}, {'Cost': 100.98}, {'Cost': 1...","[[1169270, 1169584, 1205807, 1205833, 1232292,...",[20001081]


In [458]:
df_no_index.iloc[list(neighbors[2])].head(5)

,Идентификатор СТЕ,Наименование СТЕ,Категория,Описание,Характеристики СТЕ,Регионы поставки,Кол-во заключенных контрактов,Поставщики,Страна происхождения,Другая продукция в контрактах,Просмотры,Идентификатор КПГЗ,Код КПГЗ,Модель,Цена,session_ids,item_id
2,16882580,Шеврон Кадетский,"Знаки различия наплечные, нарукавные, кокарды,...",NaN,"[{'Name': 'Материал', 'Id': -674473437, 'Value...","[{'Name': 'Москва'}, {'Name': 'Воронежская'}, ...",128.0,"[{'SupplierId': 1258604, 'Name': 'Общество с о...",РОССИЯ,"[{'OtherSkuId': 34168146, 'OtherSkuName': 'Акс...",144,67907.0,01.19.08,Шеврон Кадетское образование Москвы,"[{'Cost': 69.3}, {'Cost': 69.65}, {'Cost': 70....","[[1356954, 20219000, 34829090, 18180805, 34805...",[16882580]
1379,19744130,Пуговица форменная14мм с гербом РФ дли министе...,Одежда для детей форменная,NaN,"[{'Name': 'Вид спецодежды', 'Id': -551498056, ...","[{'Name': 'Москва'}, {'Name': 'Пермский'}, {'N...",71.0,"[{'SupplierId': 1258556, 'Name': 'Левина Алекс...",РОССИЯ,"[{'OtherSkuId': 34168146, 'OtherSkuName': 'Акс...",68,203868.0,01.19.02.17,14-2336,"[{'Cost': 6.95}, {'Cost': 7.0}, {'Cost': 8.0}]","[[22161929, 19821892, 32333183, 19538091, 2032...",[19744130]
6419,34168146,Аксельбант,"Знаки различия наплечные, нарукавные, кокарды,...",NaN,"[{'Name': 'Материал', 'Id': 339859843, 'Value'...","[{'Name': 'Москва'}, {'Name': 'Воронежская'}, ...",37.0,"[{'SupplierId': 1103468, 'Name': 'ООО ""СЕРВИСК...",NaN,"[{'OtherSkuId': 34168231, 'OtherSkuName': 'Бан...",119,67907.0,01.19.08,Аксельбант,"[{'Cost': 555.0}, {'Cost': 582.07}, {'Cost': 6...","[[21741613, 32426026, 35118239, 34838240, 2148...",[34168146]
27160,18163751,Галстук регат (синий),Одежда для взрослых форменная,NaN,"[{'Name': 'Цвет', 'Id': -644959335, 'Value': '...","[{'Name': 'Москва'}, {'Name': 'Московская'}]",7.0,"[{'SupplierId': 1286297, 'Name': 'ООО ""МИР КАМ...",NaN,"[{'OtherSkuId': 18526868, 'OtherSkuName': 'Акс...",0,67740.0,01.19.01.01,3338,[{'Cost': 121.0}],"[[22982748, 17205879, 18115025, 18165956, 1697...",[18163751]
20340,32426686,КУРТКА ФОРМЕННАЯ ПОЛУШЕРСТЯНАЯ,Одежда для детей форменная,NaN,"[{'Name': 'Описание', 'Id': -673340529, 'Value...","[{'Name': 'Москва'}, {'Name': 'Московская'}]",11.0,"[{'SupplierId': 1264150, 'Name': 'ООО ""ВОЕНСНА...",NaN,"[{'OtherSkuId': 34168146, 'OtherSkuName': 'Акс...",15,203868.0,01.19.02.17,Куртка,"[{'Cost': 2079.0}, {'Cost': 2100.0}, {'Cost': ...","[[17780408, 17032292, 32335490, 18118879, 2032...",[32426686]


In [429]:
from tqdm import tqdm
existed_similars = []
for item_similars, item_scores in tqdm(zip(neighbors, scores), total = len(neighbors)):
    similarity_score = [t for s, t in zip(item_similars, item_scores)]
    items = df_no_index.iloc[list(item_similars)][['Идентификатор СТЕ', 'Категория']]
    items['score'] = similarity_score
    #print(df_no_index[df_no_index['Идентификатор СТЕ'].isin(items['Идентификатор СТЕ'].values)])
    existed_similars.append(items.sort_values('score')['Идентификатор СТЕ'].values[1:])
results = pd.DataFrame({'Идентификатор СТЕ': df_no_index['Идентификатор СТЕ'].values, 'Сопутствующие товары':existed_similars})
results.to_csv("result_similars.csv")

100%|██████████| 49780/49780 [01:44<00:00, 474.69it/s]


In [437]:
ex = df.loc[existed_similars[1]]
ex

,Наименование СТЕ,Категория,Описание,Характеристики СТЕ,Регионы поставки,Кол-во заключенных контрактов,Поставщики,Страна происхождения,Другая продукция в контрактах,Просмотры,Идентификатор КПГЗ,Код КПГЗ,Модель,Цена,session_ids,item_id,hasSimilars,indice
Идентификатор СТЕ,,,,,,,,,,,,,,,,,,
1247785,Степлер ATTACHE 8215 (N24/6&26/6) до 25 лист. ...,Степлеры (сшиватели),NaN,"[{'Name': 'Цвет корпуса', 'Id': 299368878, 'Va...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",30.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",КИТАЙ,"[{'OtherSkuId': 1179412, 'OtherSkuName': 'Albi...",32,12104724.0,01.15.05.02.01,NaN,"[{'Cost': 155.11}, {'Cost': 184.87}, {'Cost': ...","[[1248094, 17927810, 1249120, 1237495, 1791911...",[1247785],1,738
1237057,Этикетки самоклеящиеся 3425 универсал. 105х57м...,Этикетки самоклеящиеся,NaN,"[{'Name': 'Шаблон для скачивания', 'Id': 29112...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",1.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",ГЕРМАНИЯ,"[{'OtherSkuId': 1227434, 'OtherSkuName': 'Блок...",1,12104668.0,01.15.01.09,NaN,[{'Cost': 2733.93}],"[[17872802, 1232129, 16484823, 1226949, 179535...",[1237057],1,15016
1231786,"Дырокол SAX 506 до 40л., металл, с лин., синий...",Дыроколы канцелярские,NaN,"[{'Name': 'Особенности', 'Id': 291064782, 'Val...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",5.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",NaN,"[{'OtherSkuId': 1171063, 'OtherSkuName': 'Berl...",6,12114402.0,01.15.05.02.05.01,NaN,[{'Cost': 1730.0}],"[[1232291, 1230164, 1227444, 1190254, 17796685...",[1231786],1,7906
20001081,Набор текстовыделителей Attache Economy Uno (т...,Маркеры,NaN,"[{'Name': 'Форма наконечника', 'Id': 365683510...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",90.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",КИТАЙ,"[{'OtherSkuId': 34137737, 'OtherSkuName': '""VK...",123,12104687.0,01.15.04.02.04,892707,"[{'Cost': 70.38}, {'Cost': 100.98}, {'Cost': 1...","[[1169270, 1169584, 1205807, 1205833, 1232292,...",[20001081],1,462
1231776,"Дырокол SAX 306 до 20л., металл, с лин., синий...",Дыроколы канцелярские,NaN,"[{'Name': 'Особенности', 'Id': 291064651, 'Val...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",8.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",NaN,"[{'OtherSkuId': 1182349, 'OtherSkuName': 'Mila...",0,12114402.0,01.15.05.02.05.01,NaN,[{'Cost': 967.0}],"[[1247937, 17638615, 1251068, 1237441, 1249701...",[1231776],1,6044
1227774,Файл-вкладыш Комус А4 45 мкм прозрачный рифлен...,Файлы-вкладыши пластиковые,NaN,"[{'Name': 'Материал', 'Id': 366232782, 'Value'...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",124.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",РОССИЯ,"[{'OtherSkuId': 17607484, 'OtherSkuName': '3-х...",134,12114411.0,01.15.03.17.06,162440,"[{'Cost': 214.08}, {'Cost': 219.72}, {'Cost': ...","[[1206058, 1227463, 1191317, 1251912, 34241889...",[1227774],1,326
1197329,"Стержень шариковый PARKER (Франция) ""Quinkflow...",Стержни для ручек канцелярских,NaN,"[{'Name': 'Цвет чернил', 'Id': 285964047, 'Val...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",7.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",NaN,"[{'OtherSkuId': 1251912, 'OtherSkuName': 'Анти...",1,12108486.0,01.15.04.01.09,NaN,"[{'Cost': 526.5}, {'Cost': 585.0}]","[[1247922, 21564076, 1227204, 1195848, 1253086...",[1197329],1,1627
1229896,"Точилка Kores двойная с контейнером, '35800",Точилки для карандашей,NaN,"[{'Name': 'Страна происхождения', 'Id': 291041...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",25.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",NaN,"[{'OtherSkuId': 1190892, 'OtherSkuName': 'Fell...",17,12108433.0,01.15.05.10,,"[{'Cost': 141.0}, {'Cost': 141.63}]","[[1248054, 1249800, 17919119, 1236739, 1248084...",[1229896],1,3141
1397286,Фломастеры №1 School Живая природа 12 цветов,Фломастеры,Фломастеры №1 School Живая природа 12 цветов,"[{'Name': 'Количество цветов', 'Id': 325338844...","[{'Na

In [432]:
items_data = pd.read_csv("New_table.csv", index_col=0)
items_data = items_data.merge(df.reset_index()[['Идентификатор СТЕ', 'Наименование СТЕ', 'Поставщики']])
items_data['Число поставщиков'] = items_data['Поставщики'].apply(lambda x: len(x))
ex_with_features = items_data.set_index('Идентификатор СТЕ').loc[existed_similars[1]]
ex_with_features = ex_with_features.sort_values('Просмотры')
ex_with_features['Просмотры ранк'] = np.linspace(0, 1, ex_with_features.shape[0])
ex_with_features = ex_with_features.sort_values('Число поставщиков')
ex_with_features['Число поставщиков ранк'] = np.linspace(0, 1, ex_with_features.shape[0])
ex_with_features = ex_with_features.sort_values('Кол-во заключенных контрактов')
ex_with_features['Кол-во заключенных контрактов ранк'] = np.linspace(0, 1, ex_with_features.shape[0])
ex_with_features['total_rank'] = ex_with_features['Просмотры ранк'] + ex_with_features['Число поставщиков ранк'] + ex_with_features['Кол-во заключенных контрактов ранк']
ex_with_features.sort_values('total_rank', ascending = False).head(10)

,Кол-во заключенных контрактов,Код КПГЗ_4,Новизна,Страна происхождения,Просмотры,Наименование СТЕ,Поставщики,Число поставщиков,Просмотры ранк,Число поставщиков ранк,Кол-во заключенных контрактов ранк,total_rank
Идентификатор СТЕ,,,,,,,,,,,,
1227774,124.0,01.15.03.17.06,0,РОССИЯ,134,Файл-вкладыш Комус А4 45 мкм прозрачный рифлен...,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",52,1.000000,1.000000,1.000000,3.000000
20001081,90.0,01.15.04.02.04,0,КИТАЙ,123,Набор текстовыделителей Attache Economy Uno (т...,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",40,0.944444,0.944444,0.944444,2.833333
1232160,41.0,01.15.05.04.05,0,КИТАЙ,26,"Кнопки Attache 12 мм, цветные 50 штук пластико...","[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",25,0.833333,0.833333,0.833333,2.500000
1247785,30.0,01.15.05.02.01,0,КИТАЙ,32,Степлер ATTACHE 8215 (N24/6&26/6) до 25 лист. ...,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",23,0.888889,0.777778,0.777778,2.444444
1240833,48.0,01.15.05.18,0,РОССИЯ,12,Линейка металлическая 50 см Attache,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",38,0.611111,0.888889,0.888889,2.388889
1248594,30.0,01.15.05.04.03,0,ЧЕШСКАЯ РЕСПУБЛИКА,24,Корректирующая лента ATTACHE 20м '831,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",23,0.777778,0.722222,0.722222,2.222222
1229896,25.0,01.15.05.10,0,NaN,17,"Точилка Kores двойная с контейнером, '35800","[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",21,0.722222,0.555556,0.666667,1.944444
1248330,15.0,01.15.04.03.04,0,ЯПОНИЯ,14,"Стержень микрографический 0,7 Attache HB 30 гр...","[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",19,0.666667,0.500000,0.500000,1.666667
1229898,24.0,01.15.05.10,0,NaN,3,Точилка MAPED Shaker 1 отв. с конт. '534753,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",21,0.333333,0.666667,0.611111,1.611111


In [503]:
import faiss
from scipy import sparse
import numpy as np

vectors_sparse = sparse.load_npz('vectors.npz')
embeddings_tfidf = np.array(vectors_sparse.todense())

def prepare_tfidf_embeddings(df, existed_index, embeddings):
    df['hasSimilars'] = 0
    df.loc[existed_index, 'hasSimilars'] = 1
    df['indice'] = range(df.shape[0])
    drop_indices = df[df['hasSimilars'] == 0]['indice'].values
    use_indices = df[df['hasSimilars'] == 1]['indice'].values

    embeddings_tfidf_non_target = embeddings_tfidf[drop_indices]
    embeddings_tfidf_target = embeddings_tfidf[use_indices]
    return embeddings_tfidf_non_target, embeddings_tfidf_target

embeddings_tfidf_non_target, embeddings_tfidf_target = prepare_tfidf_embeddings(df, existed_index, embeddings)
def fit_search_index(vectors):
    index = faiss.index_factory(5000, "IVF100,Flat")

    index.train(vectors) # вектора, которые могут быть соседями
    index.add(vectors) # вектора, которые могут быть соседями
    return index


def similars(index, vectors, top_k):
    scores, neighbors = index.search(vectors, top_k)
    return neighbors, scores


def find_neighbors_by_tfidf_neighbor(item_tfidf_index):
    neighbors_from, _ = similars(index, embeddings_tfidf.astype(np.float32)[item_tfidf_index:item_tfidf_index+1], 1)
    similar_item_ind = ste_to_similar_index[df[df['hasSimilars'] == 1].reset_index().iloc[neighbors_from[0]]['Идентификатор СТЕ'].values[0]]
    return neighbors[similar_item_ind]

def simple_index(ste):
    return df.loc[ste]['indice']

#neighbors_tfidf, _ = similars(index, embeddings_tfidf_non_target.astype(np.float32)[:100], 1)
def predict_by_w2v(ste):
    true_index = ste_to_similar_index[ste]
    return df_no_index.iloc[list(neighbors[true_index])].head(5)
def predict_by_tfidf(ste):
    df[df['hasSimilars'] == 1].reset_index().loc[find_neighbors_by_tfidf_neighbor(simple_index(ste))]
def predict(ste):
    try:
        return predict_by_w2v(ste)
    except Exception:
        print("No w2v vector")
        return predict_by_tfidf(ste)
index = fit_search_index(embeddings_tfidf_target.astype(np.float32))


In [504]:
predict(1243489)

,Идентификатор СТЕ,Наименование СТЕ,Категория,Описание,Характеристики СТЕ,Регионы поставки,Кол-во заключенных контрактов,Поставщики,Страна происхождения,Другая продукция в контрактах,Просмотры,Идентификатор КПГЗ,Код КПГЗ,Модель,Цена,session_ids,item_id
2394,1243489,Папка для черчения А4 24л. ГОЗНАК ватман 200гр...,Папки для черчения,NaN,"[{'Name': 'Формат листа', 'Id': 291189015, 'Va...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",17.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",NaN,"[{'OtherSkuId': 1189439, 'OtherSkuName': 'Brau...",29,12106737.0,01.15.07.02,NaN,"[{'Cost': 133.74}, {'Cost': 152.03}, {'Cost': ...","[[1253079, 1243375, 1205825, 28530370, 1229929...",[1243489]
6490,1214909,"Обложки для переплета GBC (Англия), комплект 1...",Обложки для переплета пластиковые,NaN,"[{'Name': 'Цвет', 'Id': 286337406, 'Value': 'ч...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",2.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",NaN,"[{'OtherSkuId': 1179100, 'OtherSkuName': 'Pala...",0,12120455.0,01.15.05.02.06.01.02,NaN,[{'Cost': 2920.0}],"[[1210557, 1214909, 1406143, 1210549, 1210554,...",[1214909]
34814,23846680,"Универсальный клей UHU TWIST&GLUE, 90 мл",Клеи канцелярские,"Универсальный клей Uhu ""Twist and Glue"" - быст...","[{'Name': 'Тип', 'Id': -229384619, 'Value': 'К...","[{'Name': 'Москва'}, {'Name': 'Московская'}]",2.0,"[{'SupplierId': 1105608, 'Name': 'Общество с о...",NaN,"[{'OtherSkuId': 33686512, 'OtherSkuName': 'Ske...",3,12106696.0,01.15.05.04.04,"Twist & Glue, 90 мл",None,"[[1250640, 20356608, 1252482, 23846680, 124066...",[23846680]
37455,23107049,Доска-планшет с зажимом,Папки пластиковые,NaN,"[{'Name': 'Цвет', 'Id': -320396418, 'Value': '...","[{'Name': 'Москва'}, {'Name': 'Пермский'}]",4.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",NaN,"[{'OtherSkuId': 34294668, 'OtherSkuName': '""IN...",5,12114414.0,01.15.03.17.05,2,"[{'Cost': 57.0}, {'Cost': 100.0}]","[[23274350, 1247911, 1186811, 1182500, 1972883...",[23107049]
38986,19423696,"Ластик Berlingo ""Blitz"", скошенный, комбиниров...",Ластики,NaN,"[{'Name': 'Вид письменных принадлежностей', 'I...",[{'Name': 'Москва'}],3.0,"[{'SupplierId': 1107637, 'Name': 'ООО ""Алком-М...",МАЛАЙЗИЯ,"[{'OtherSkuId': 1170064, 'OtherSkuName': 'Crow...",11,12108430.0,01.15.04.07,"Ластик Berlingo ""Blitz""",None,"[[19327991, 19432492, 1189435, 19322385, 19027...",[19423696]


In [485]:
predict_by_w2v(1227774)

,Наименование СТЕ,Категория,Описание,Характеристики СТЕ,Регионы поставки,Кол-во заключенных контрактов,Поставщики,Страна происхождения,Другая продукция в контрактах,Просмотры,Идентификатор КПГЗ,Код КПГЗ,Модель,Цена,session_ids,item_id,hasSimilars,indice
Идентификатор СТЕ,,,,,,,,,,,,,,,,,,
1227774,Файл-вкладыш Комус А4 45 мкм прозрачный рифлен...,Файлы-вкладыши пластиковые,NaN,"[{'Name': 'Материал', 'Id': 366232782, 'Value'...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",124.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",РОССИЯ,"[{'OtherSkuId': 17607484, 'OtherSkuName': '3-х...",134,12114411.0,01.15.03.17.06,162440,"[{'Cost': 214.08}, {'Cost': 219.72}, {'Cost': ...","[[1206058, 1227463, 1191317, 1251912, 34241889...",[1227774],1,326
1221127,"Батарейки аккумуляторные GP, AAA, Ni-Mh, 850 m...",Батареи аккумуляторные никель-металл-гибридные,NaN,"[{'Name': 'Типоразмер', 'Id': 287129501, 'Valu...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",14.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",NaN,"[{'OtherSkuId': 34733878, 'OtherSkuName': ' Ба...",44,12121754.0,01.10.04.03.03,NaN,[{'Cost': 379.68}],"[[1238606, 23765175, 1220904, 17894251, 343144...",[1221127],0,3390
1256411,Флеш-память Smartbuy 16GB Crown Black,USB-накопители твердотельные (флеш-драйвы),NaN,"[{'Name': 'Интерфейс', 'Id': 299326179, 'Value...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",23.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",КИТАЙ,"[{'OtherSkuId': 17109951, 'OtherSkuName': 'AA ...",19,12106800.0,01.13.05.06.02,Crown Black,[{'Cost': 321.0}],"[[1332477, 1232153, 19432490, 1205830, 1230602...",[1256411],0,6552
1237141,"Этикетки самоклеящиеся Europe ELA001-18, 38x21...",Этикетки самоклеящиеся,NaN,"[{'Name': 'Шаблон для скачивания', 'Id': 29112...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",2.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",ГЕРМАНИЯ,"[{'OtherSkuId': 1236752, 'OtherSkuName': 'Бизн...",0,12104668.0,01.15.01.09,NaN,[{'Cost': 186.69}],"[[1251447, 34135120, 1237141, 1250840, 1232358...",[1237141],1,15580
17949361,"Тетрадь общая А5,48л,клет,скреп,обл.бумвинил,б...",Тетрадь ученическая общая,NaN,"[{'Name': 'Материал обложки', 'Id': -653645601...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",10.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",РОССИЯ,"[{'OtherSkuId': 1169308, 'OtherSkuName': 'Beif...",1,1100146.0,01.15.02.06.02,NaN,[{'Cost': 27.62}],"[[1241005, 1231813, 1240820, 1233091, 1232558,...",[17949361],1,11961
1232324,Блок-кубик Post-it 654-OGP 76х76 весна плюс 12...,Блоки бумажные для записей,NaN,"[{'Name': 'Тон стикеров', 'Id': 291069382, 'Va...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",4.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",РОССИЯ,"[{'OtherSkuId': 1182349, 'OtherSkuName': 'Mila...",0,66915.0,01.15.01.07,NaN,[{'Cost': 195.64}],"[[22056327, 1444022, 1227772, 1231594, 1248385...",[1232324],1,11282
1218967,Кисть художественная ROUBLOFF (Рублев) белка м...,Кисти художественные,NaN,"[{'Name': 'Упаковка', 'Id': 286348835, 'Value'...","[{'Name': 'Москва'}, {'Name': 'Пермский'}, {'N...",3.0,"[{'SupplierId': 1103258, 'Name': 'ООО ""ОФИСМАГ...",NaN,"[{'OtherSkuId': 22981098, 'OtherSkuName': 'Акр...",7,12104871.0,01.07.05.02,NaN,None,"[[22980541, 1219661, 23046764, 18751590, 12444...",[1218967],0,16515
1243489,Папка для черчения А4 24л. ГОЗНАК ватман 200гр...,Папки для черчения,NaN,"[{'Name': 'Формат листа', 'Id': 291189015, 'Va...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",17.0,"[{'SupplierId': 1100019, 'Name': 'ООО ""ЦТО""', ...",NaN,"[{'OtherSkuId': 1189439, 'OtherSkuName': 'Brau...",29,12106737.0,01.15.07.02,NaN,"[{'Cost': 133.74}, {'Cost': 152.03}, {'Cost': ...","[[1253079, 1243375, 1205825, 28530370, 1229929...",[1243489],1,4379
1226518,Метал.Мебель P_КБC 10 шкаф д/бумаг 880х390х1850,Шкафы архивные металлические,NaN,"[{'Name': 'Тип замка', 'Id': 291003119, 'Value...","[{'Name': 'Белгородская'}, {'Name': 'Марий Эл'...",4.0,"[{'Suppli